In [ ]:
# !pip install skrebate

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import gc
import os
import sys

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

In [3]:
seed = 100
random_state = seed

In [7]:
def feature_importance_reliefF(df):
    X = df.drop(columns=['label'])
    y = df['label']
    genes = X.columns
    X = X.to_numpy()
    y = y.to_numpy()
    
    relief = ReliefF(n_neighbors=100, verbose=True)  # n_neighbors can be tuned
    relief.fit(X, y)
    
    # Get feature scores
    feature_scores = relief.feature_importances_
    # Map scores to gene names
    gene_importance = pd.DataFrame({
        "Gene": genes,
        "Importance": feature_scores
    })
    return gene_importance

---

In [5]:
rna_luad_xena = pd.read_csv(os.path.join('Z:\multiomics based manuscript\datasets\gene_exp\with_common_patients_processed', 'csv_rna_common_luad.csv'))
rna_lusc_xena = pd.read_csv(os.path.join('Z:\multiomics based manuscript\datasets\gene_exp\with_common_patients_processed', 'csv_rna_common_lusc.csv'))

rna_luad_xena['label'] = 1
rna_lusc_xena['label'] = 0
df_rna_xena = pd.concat([rna_luad_xena, rna_lusc_xena], axis=0)

cnv_luad_xena = pd.read_csv(os.path.join('Z:\multiomics based manuscript\datasets\cnv\with_common_patients_processed', 'csv_cnv_common_luad.csv'))
cnv_lusc_xena = pd.read_csv(os.path.join('Z:\multiomics based manuscript\datasets\cnv\with_common_patients_processed', 'csv_cnv_common_lusc.csv'))

cnv_luad_xena['label'] = 1
cnv_lusc_xena['label'] = 0
df_cnv_xena = pd.concat([cnv_luad_xena, cnv_lusc_xena], axis=0)

In [6]:
_df_rna_xena = df_rna_xena.sample(frac=1, replace=False, ignore_index=True, random_state = seed)
_df_cnv_xena = df_cnv_xena.sample(frac=1, replace=False, ignore_index=True, random_state = seed)

In [8]:
'''
Feature importance using ReliefF
'''

from skrebate import ReliefF

reliefF_feature_importance_rna = feature_importance_reliefF(_df_rna_xena)
print('Done with RNA.')
reliefF_feature_importance_cnv = feature_importance_reliefF(_df_cnv_xena)
print('Done with CNV.')

Created distance array in 40.55929899215698 seconds.
Feature scoring under way ...
Completed scoring in 3288.8320817947388 seconds.
Done with RNA.
Created distance array in 51.25565719604492 seconds.
Feature scoring under way ...
Completed scoring in 3987.9741406440735 seconds.
Done with CNV.


In [9]:
IMP_GENES_PATH = 'Z:/multiomics based manuscript/results_for_xena_rna_cnv_only/important_genes'

reliefF_feature_importance_rna = reliefF_feature_importance_rna.sort_values(by='Importance', ascending=False).reset_index(drop=True)
reliefF_feature_importance_rna.to_csv(os.path.join(IMP_GENES_PATH, 'relieff_feature_importance_rna_df.csv'), index=False)

reliefF_feature_importance_cnv = reliefF_feature_importance_cnv.sort_values(by='Importance', ascending=False).reset_index(drop=True)
reliefF_feature_importance_cnv.to_csv(os.path.join(IMP_GENES_PATH, 'relieff_feature_importance_cnv_df.csv'), index=False)


In [ ]:
# plt.figure(figsize=(10, 50))
# plt.barh(importance_df["Gene"], importance_df["Importance"], color="steelblue")
# plt.xlabel("Importance Score (Gain)")
# plt.ylabel("Gene")
# plt.title("Top 10 Important Genes Based on XGBoost")
# plt.gca().invert_yaxis()  # Highest importance at the top
# plt.show()